In [1]:
from pyspark.sql import SparkSession, SQLContext, HiveContext
from pyspark import SparkConf, SparkContext
from datetime import datetime
import numpy as np
import pandas as pd
import json
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import Row
import sys #这里只是一个对sys的引用，只能reload才能进行重新加载
stdi,stdo,stde=sys.stdin,sys.stdout,sys.stderr 
reload(sys) #通过import引用进来时,setdefaultencoding函数在被系统调用后被删除了，所以必须reload一次
sys.stdin,sys.stdout,sys.stderr=stdi,stdo,stde 
sys.setdefaultencoding('utf-8')
#spark.stop()
spark = SparkSession.builder.appName('jiahui_chen').config('spark.debug.maxToStringFields', '100').enableHiveSupport().getOrCreate()
sc=SparkContext.getOrCreate()

**读取文件，读取的是rdd形式**

In [2]:
rdd = sc.parallelize([1,2,3,4,5])

**查看分区数**

In [3]:
rdd.getNumPartitions()

1000

**看看每个文件存在哪个区了**

In [4]:
rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],

# --transformation操作--

**map():** 对RDD的每一个item都执行同一个操作

**flatMap():** 对RDD中的item执行同一个操作以后得到一个list，然后以平铺的方式把这些list里所有的结果组成新的list

**filter():** 筛选出来满足条件的item

**distinct():** 对RDD中的item去重

**sample():** 从RDD中的item中采样一部分出来，有放回或者无放回

**sortBy():** 对RDD中的item进行排序


## map() 

In [5]:
numbersRDD = sc.parallelize(range(1,10+1))
print(numbersRDD.collect())

#map()对RDD的每一个item都执行同一个操作
squaresRDD = numbersRDD.map(lambda x: x**2)  # Square every number
print(squaresRDD.collect())

#filter()筛选出来满足条件的item
filteredRDD = numbersRDD.filter(lambda x: x % 2 == 0)  # Only the evens
print(filteredRDD.collect())



[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[2, 4, 6, 8, 10]


## flatMap() 

In [6]:
sentencesRDD=sc.parallelize(['Hello world','My name is Patrick'])
wordsRDD_map=sentencesRDD.map(lambda sentence: sentence.split(" "))
print(wordsRDD_map.collect())
print(wordsRDD_map.count())

wordsRDD=sentencesRDD.flatMap(lambda sentence: sentence.split(" "))
print(wordsRDD.collect())
print(wordsRDD.count())


[['Hello', 'world'], ['My', 'name', 'is', 'Patrick']]
2
['Hello', 'world', 'My', 'name', 'is', 'Patrick']
6


In [7]:
def doubleIfOdd(x):
    if x % 2 == 1:
        return 2 * x
    else:
        return x
numbersRDD = sc.parallelize(range(1,10+1))
resultRDD = (numbersRDD          
             .map(doubleIfOdd)    #map,filter,distinct()
             .filter(lambda x: x > 6)
             .distinct())    #distinct()对RDD中的item去重
resultRDD.collect()


[8, 10, 14, 18]

## 当遇到更复杂的结构，比如被称作“pair RDDs”的以元组形式组织的k-v对（key, value），Spark中针对这种item结构的数据，定义了一些transform和action:

**reduceByKey():** 对所有有着相同key的items执行reduce操作

**groupByKey():** 返回类似(key, listOfValues)元组的RDD，后面的value List 是同一个key下面的

**sortByKey():** 按照key排序

**countByKey():** 按照key去对item个数进行统计

**collectAsMap():** 和collect有些类似，但是返回的是k-v的字典


In [8]:
rdd=sc.parallelize(["Hello hello", "Hello New York", "York says hello"])

resultRDD=(rdd
    .flatMap(lambda sentence:sentence.split(" "))  
    .map(lambda word:word.lower())                 
    .map(lambda word:(word, 1))    #将word映射成(word,1)                  
    .reduceByKey(lambda x, y: x + y))  #reduceByKey对所有有着相同key的items执行reduce操作

print resultRDD.collect()

#Output:[('hello', 4), ('york', 2), ('says', 1), ('new', 1)]

result = resultRDD.collectAsMap()  #collectAsMap类似collect,以k-v字典的形式返回

print result

#Output:{'hello': 4, 'new': 1, 'says': 1, 'york': 2}

resultRDD.sortByKey(ascending=True).take(4) #sortByKey按键排序

#Output:[('hello', 4), ('new', 1)]

#取出现频次最高的2个词
print(resultRDD
      .sortBy(lambda x: x[1], ascending=False)
      .take(2))


[('new', 1), ('says', 1), ('york', 2), ('hello', 4)]
{'new': 1, 'says': 1, 'york': 2, 'hello': 4}
[('hello', 4), ('york', 2)]


# 惰性计算，actions方法

**collect():** 计算所有的items并返回所有的结果到driver端，接着 collect()会以Python list的形式返回结果

**first():** 和上面是类似的，不过只返回第1个item

**take(n):** 类似，但是返回前n个item

**count():** 计算RDD中item的个数

**top(n):** 返回头n个items，按照自然结果排序

**reduce():** 对RDD中的items做聚合

In [9]:
rdd = sc.parallelize(range(1,10+1))
rdd.reduce(lambda x, y: x + y)  #reduce(): 对RDD中的items做聚合

55

# spark dataframe操作方法

## 创建spark dataframe

In [43]:
data = [(123, "Katie", 19, "brown"),
        (234, "Michael", 22, "green"),
        (345, "Simone", 23, "blue"),
       (None,'Cathy' ,None ,'red'),
       (143,None,15,None)]
df = spark.createDataFrame(data, schema=['id', 'name', 'age', 'eyccolor'])
df.show()
df.count()


+----+-------+----+--------+
|  id|   name| age|eyccolor|
+----+-------+----+--------+
| 123|  Katie|  19|   brown|
| 234|Michael|  22|   green|
| 345| Simone|  23|    blue|
|null|  Cathy|null|     red|
| 143|   null|  15|    null|
+----+-------+----+--------+



5

**从 pandas dataframe 转换**

In [10]:
iris=pd.read_csv('iris.csv')
iris.drop('Unnamed: 0',axis=1,inplace=True)
iris=spark.createDataFrame(iris)

## spark df 的特殊结构 row 与colomn

In [11]:
iris.first()

Row(Sepal.Length=5.1, Sepal.Width=3.5, Petal.Length=1.4, Petal.Width=0.2, Species=u'setosa')

In [12]:
iris.columns

['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']

In [13]:
iris['`Sepal.Length`']

Column<Sepal.Length>

## 指定数据类型读取

In [14]:
iris.show(3)

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 3 rows



## 行元素操作

**以树的形式打印数据概要**

In [15]:
iris.printSchema()

root
 |-- Sepal.Length: double (nullable = true)
 |-- Sepal.Width: double (nullable = true)
 |-- Petal.Length: double (nullable = true)
 |-- Petal.Width: double (nullable = true)
 |-- Species: string (nullable = true)



**获取头几行到本地**

In [16]:
iris_list = iris.take(5)

**查询总行数：**

In [17]:
iris.count()

150

**按条件筛选行**

In [18]:
iris.filter(iris['`Sepal.Length`']>=5).show()

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         5.8|        4.0|         1.2|        0.2| setosa|
|         5.7|        4.4|         1.5|        0.4| setosa|
|         5.4|        3.9|         1.3|        0.4| setosa|
|         5.1|        3.5|         1.4|        0.3| setosa|
|         5.7|        3.8|         1.7|        0.3| setosa|
|         5.1|        3.8|         1.5|        0.3| setosa|
|         5.4|        3.4|         1.7|        0.2| setosa|
|         5.1|        3.7|         1.5|        0.4| setosa|
|         5.1|        3.3|         1.7| 

In [46]:
from pyspark.sql.functions import isnull
df.filter(isnull(df['name'])).show()

+---+----+---+--------+
| id|name|age|eyccolor|
+---+----+---+--------+
|143|null| 15|    null|
+---+----+---+--------+



**还可以用where按条件选择**

In [19]:
iris.where((iris['`Sepal.Length`']>=5) & (iris['`Sepal.Width`']<=4)).show()

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         5.8|        4.0|         1.2|        0.2| setosa|
|         5.4|        3.9|         1.3|        0.4| setosa|
|         5.1|        3.5|         1.4|        0.3| setosa|
|         5.7|        3.8|         1.7|        0.3| setosa|
|         5.1|        3.8|         1.5|        0.3| setosa|
|         5.4|        3.4|         1.7|        0.2| setosa|
|         5.1|        3.7|         1.5|        0.4| setosa|
|         5.1|        3.3|         1.7|        0.5| setosa|
|         5.0|        3.0|         1.6| 

## 列元素操作 

**取列名**

In [20]:
iris.columns

['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']

**以datafram的形式取列**

In [21]:
Length=iris.select('`Sepal.Length`')
print type(Length)

<class 'pyspark.sql.dataframe.DataFrame'>


##  排序 

In [22]:
iris.orderBy(iris['`Sepal.Length`'].desc()).show(5)

+------------+-----------+------------+-----------+---------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|  Species|
+------------+-----------+------------+-----------+---------+
|         7.9|        3.8|         6.4|        2.0|virginica|
|         7.7|        3.8|         6.7|        2.2|virginica|
|         7.7|        2.6|         6.9|        2.3|virginica|
|         7.7|        2.8|         6.7|        2.0|virginica|
|         7.7|        3.0|         6.1|        2.3|virginica|
+------------+-----------+------------+-----------+---------+
only showing top 5 rows



## 抽样

In [23]:
iris.sample(withReplacement=False,fraction=0.2, seed=1).count()

27

## 增加新列

**不生成新列**

In [24]:
iris.withColumn('new_columns',iris['`Petal.Width`']*2)

iris.show(2)

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 2 rows



In [25]:
from pyspark.sql import functions
iris = iris.withColumn('label',  functions.lit(0))

**生成新列**

In [26]:
iris.show(2)

+------------+-----------+------------+-----------+-------+-----+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|label|
+------------+-----------+------------+-----------+-------+-----+
|         5.1|        3.5|         1.4|        0.2| setosa|    0|
|         4.9|        3.0|         1.4|        0.2| setosa|    0|
+------------+-----------+------------+-----------+-------+-----+
only showing top 2 rows



**想要新的变量出现只有赋值给新变量或者覆盖**

In [31]:
def double_x(x):
    return x*2

iris_new=iris.withColumn('new_columns',double_x(iris['`Petal.Width`']))

In [32]:
iris_new.show(2)

+------------+-----------+------------+-----------+-------+-----+-----------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|label|new_columns|
+------------+-----------+------------+-----------+-------+-----+-----------+
|         5.1|        3.5|         1.4|        0.2| setosa|    0|        0.4|
|         4.9|        3.0|         1.4|        0.2| setosa|    0|        0.4|
+------------+-----------+------------+-----------+-------+-----+-----------+
only showing top 2 rows



**修改列名**

In [33]:
iris_new.withColumnRenamed( "new_columns" , "new_column" )

DataFrame[Sepal.Length: double, Sepal.Width: double, Petal.Length: double, Petal.Width: double, Species: string, label: int, new_column: double]

## 分组统计

In [34]:
iris_new.groupby('`Petal.Width`').count().show()

+-----------+-----+
|Petal.Width|count|
+-----------+-----+
|        2.4|    3|
|        1.3|   13|
|        0.4|    7|
|        1.6|    4|
|        1.7|    2|
|        1.1|    3|
|        2.5|    3|
|        0.6|    1|
|        2.2|    3|
|        0.5|    1|
|        0.2|   29|
|        2.3|    8|
|        1.8|   12|
|        1.0|    7|
|        0.3|    7|
|        2.1|    6|
|        1.4|    8|
|        2.0|    6|
|        0.1|    5|
|        1.2|    5|
+-----------+-----+
only showing top 20 rows



In [36]:
iris_new.groupby('`Petal.Width`').agg({'`Sepal.Width`': 'mean'}).show()

+-----------+------------------+
|Petal.Width|  avg(Sepal.Width)|
+-----------+------------------+
|        2.4|3.0999999999999996|
|        1.3|2.7461538461538457|
|        0.4|3.7857142857142856|
|        1.6|               3.1|
|        1.7|              2.75|
|        1.1| 2.466666666666667|
|        2.5|               3.4|
|        0.6|               3.5|
|        2.2|3.1999999999999997|
|        0.5|               3.3|
|        0.2|3.3793103448275863|
|        2.3|3.0874999999999995|
|        1.8|2.9416666666666664|
|        1.0| 2.371428571428571|
|        0.3|3.3285714285714287|
|        2.1| 3.033333333333333|
|        1.4|            2.9125|
|        2.0| 3.016666666666667|
|        0.1|3.3599999999999994|
|        1.2|2.7399999999999998|
+-----------+------------------+
only showing top 20 rows



In [37]:
iris_new.groupBy('`Petal.Width`').agg(    
    functions.avg('`Sepal.Width`'),
    functions.min('`Sepal.Length`'), 
    functions.max('`Petal.Length`')).show()

+-----------+------------------+-------------------+-------------------+
|Petal.Width|avg(`Sepal.Width`)|min(`Sepal.Length`)|max(`Petal.Length`)|
+-----------+------------------+-------------------+-------------------+
|        2.4|               3.1|                5.8|                5.6|
|        1.3|2.7461538461538457|                5.5|                4.6|
|        0.4|3.7857142857142856|                5.0|                1.9|
|        1.6|3.0999999999999996|                6.0|                5.8|
|        1.7|              2.75|                4.9|                5.0|
|        1.1| 2.466666666666667|                5.1|                3.9|
|        2.5|               3.4|                6.3|                6.1|
|        0.6|               3.5|                5.0|                1.6|
|        2.2|3.1999999999999997|                6.4|                6.7|
|        0.5|               3.3|                5.1|                1.7|
|        0.2|3.3793103448275863|                4.4

## 去空值和重复值

In [47]:
iris_new.select('`Sepal.Width`').dropDuplicates().show()

+-----------+
|Sepal.Width|
+-----------+
|        2.4|
|        4.2|
|        2.9|
|        2.5|
|        4.4|
|        2.2|
|        3.6|
|        3.3|
|        3.8|
|        3.5|
|        3.7|
|        3.0|
|        2.3|
|        2.6|
|        4.1|
|        2.8|
|        4.0|
|        3.2|
|        2.0|
|        3.4|
+-----------+
only showing top 20 rows



In [49]:
df.select('name').dropna().show()

+-------+
|   name|
+-------+
|  Katie|
|Michael|
| Simone|
|  Cathy|
+-------+



In [50]:
df.na.drop().show()

+---+-------+---+--------+
| id|   name|age|eyccolor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [51]:
df = df.dropna(subset=['name', 'eyccolor']) 

In [52]:
df.show()

+----+-------+----+--------+
|  id|   name| age|eyccolor|
+----+-------+----+--------+
| 123|  Katie|  19|   brown|
| 234|Michael|  22|   green|
| 345| Simone|  23|    blue|
|null|  Cathy|null|     red|
+----+-------+----+--------+



In [53]:
df.fillna(0).show()

+---+-------+---+--------+
| id|   name|age|eyccolor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
|  0|  Cathy|  0|     red|
+---+-------+---+--------+

